In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import math

In [2]:
class Lattice:
    def printLattice(self):
        for t, level in enumerate(self.lattice):
            print 'level {0}'.format(t)
            level = [ round(elem, 3) for elem in level ]
            print ', '.join(map(str, level)) 

In [3]:
class PriceLattice(Lattice):
    def __init__(self, n, S0, u, d):
        self.lattice = []
        for i in range(n+1):
            level = []
            for j in range(i+1):
                price = S0 * u**j * d**(i - j)
                level.append(price)
            self.lattice.append(level)

In [4]:
class OptionLattice(Lattice):
    def __init__(self, n, u, d, K, R, isCall, isAmerican, c, baseLattice):
        q = (R-d-c)/(u-d)
        multiplier = 1 if isCall else -1
        self.lattice = []
        clippedBase = baseLattice[:n+1]
        rightLevel = []
        print "Calculating options on prices"
        for i, level in enumerate(reversed(clippedBase)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    newLevel.append(max(multiplier * (level[j]-K), 0))
            else:
                for j in range(len(level)):
                    earlyExercise = max(multiplier * (level[j]-K), 0)
                    hold = (q*rightLevel[j+1] + (1-q)*rightLevel[j])/R
                    if earlyExercise > hold and isAmerican:
                        print "At time {0}, it's better to early exercise {1} than hold {2}".format(len(clippedBase)-i-1, earlyExercise, hold)
                    newPrice = max(hold, earlyExercise) if isAmerican else hold
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [5]:
class FutureLattice(PriceLattice):
    def __init__(self, n, u, d, c, R, baseLattice):
        self.lattice = baseLattice[:n+1]
        q = (R-d-c)/(u-d)
        for i, level in enumerate(reversed(self.lattice)):
            if i != 0:
                rightLevel = self.lattice[len(self.lattice)-i]
                for j in range(len(level)):
                    level[j] = q*rightLevel[j+1]+(1-q)*rightLevel[j]

In [6]:
# Problem 1
numPeriods = 15
years = 0.25
startPrice = 100
strikePrice = 110
continuousInterestRate = .02
volatility = .3
isCall = True
isAmerican = True

riskFreeReturn = math.exp(continuousInterestRate*years/numPeriods)
dividendYield = .01*years/numPeriods
upMoveReturn = math.exp(volatility*math.sqrt(years/numPeriods))
downMoveReturn = 1./upMoveReturn

pL = PriceLattice(numPeriods, startPrice, upMoveReturn, downMoveReturn)
oL = OptionLattice(numPeriods, upMoveReturn, downMoveReturn, strikePrice, riskFreeReturn, isCall, isAmerican, dividendYield, pL.lattice[:])
print round(oL.lattice[0][0], 2)

Calculating options on prices
2.6


In [7]:
# Problem 2
isCall = False
oL = OptionLattice(numPeriods, upMoveReturn, downMoveReturn, strikePrice, riskFreeReturn, isCall, isAmerican, dividendYield, pL.lattice[:])
print round(oL.lattice[0][0], 2)

Calculating options on prices
At time 14, it's better to early exercise 51.8542658116 than hold 51.8272929813
At time 14, it's better to early exercise 47.1712868691 than hold 47.1450942752
At time 14, it's better to early exercise 42.111147124 than hold 42.0857976056
At time 14, it's better to early exercise 36.6434705543 than hold 36.6190320116
At time 14, it's better to early exercise 30.7354346944 than hold 30.7119804962
At time 14, it's better to early exercise 24.3515736013 than hold 24.3291830254
At time 14, it's better to early exercise 17.4535649532 than hold 17.4323236622
At time 14, it's better to early exercise 10.0 than hold 9.9800005558
At time 13, it's better to early exercise 49.558113422 than hold 49.5315231562
At time 13, it's better to early exercise 44.6902054655 than hold 44.6644262473
At time 13, it's better to early exercise 39.4302427733 than hold 39.4053399234
At time 13, it's better to early exercise 33.7466497861 than hold 33.722693886
At time 13, it's better

In [19]:
# Problem 3
# Answer: Yes

In [20]:
# Problem 4
# Asnwer: 5

In [21]:
# Problem 5
# Answer: No, put-call parity only applies for European options

In [8]:
# Problem 6
numPeriods = 15
years = 0.25
startPrice = 100
strikePrice = 110
continuousInterestRate = .02
volatility = .3
isCall = True
isAmerican = True

riskFreeReturn = math.exp(continuousInterestRate*years/numPeriods)
dividendYield = .01*years/numPeriods
upMoveReturn = math.exp(volatility*math.sqrt(years/numPeriods))
downMoveReturn = 1./upMoveReturn

pL = PriceLattice(numPeriods, startPrice, upMoveReturn, downMoveReturn)
fL = FutureLattice(numPeriods, upMoveReturn, downMoveReturn, dividendYield, riskFreeReturn, pL.lattice)

numPeriods = 10
oL = OptionLattice(numPeriods, upMoveReturn, downMoveReturn, strikePrice, riskFreeReturn, isCall, isAmerican, dividendYield, fL.lattice[:])

print round(oL.lattice[0][0], 2)

Calculating options on prices
At time 9, it's better to early exercise 11.4884994161 than hold 11.4846705545
At time 9, it's better to early exercise 21.2730191657 than hold 21.265929341
At time 9, it's better to early exercise 31.8455709281 than hold 31.8349575068
At time 8, it's better to early exercise 16.3070876151 than hold 16.3016528251
At time 8, it's better to early exercise 26.4796899538 than hold 26.4708648614
At time 7, it's better to early exercise 21.3167950752 than hold 21.3096906609
1.66


In [25]:
# Problem 7
# Answer: 7

In [9]:
# Problem 8
numPeriodsExpiration = 15
years = 0.25

continuousInterestRate = .02
volatility = .3
isAmerican = False

isCall = True
startPrice = 100
strikePrice = 100
riskFreeReturn = math.exp(continuousInterestRate*years/numPeriodsExpiration)
dividendYield = .01*years/numPeriodsExpiration
upMoveReturn = math.exp(volatility*math.sqrt(years/numPeriodsExpiration))
downMoveReturn = 1./upMoveReturn

pL = PriceLattice(numPeriodsExpiration, startPrice, upMoveReturn, downMoveReturn)
callL = OptionLattice(numPeriodsExpiration, upMoveReturn, downMoveReturn, strikePrice, riskFreeReturn, isCall, isAmerican, dividendYield, pL.lattice[:])

isCall = False
putL = OptionLattice(numPeriodsExpiration, upMoveReturn, downMoveReturn, strikePrice, riskFreeReturn, isCall, isAmerican, dividendYield, pL.lattice[:])

numPeriodsChoose = 10

chooserL = []
rightLevel = []
q = (riskFreeReturn - downMoveReturn - dividendYield) / (upMoveReturn - downMoveReturn)

for i in reversed(range(numPeriodsChoose+1)):
    newLevel = []
    if i == numPeriodsChoose:
        newLevel = [max(callL.lattice[i][j],putL.lattice[i][j]) for j in range(i+1)]
    else:
        for j in range(i+1):
            price = (q*rightLevel[j+1] + (1-q)*rightLevel[j])/riskFreeReturn
            newLevel.append(price)
    rightLevel = newLevel
    chooserL.insert(0, newLevel)

print round(chooserL[0][0], 2)

Calculating options on prices
Calculating options on prices
10.81
